In [ ]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,8327.55,-0.13,-0.03,-0.02,0.00,-0.00,0.0,-0.00
3,-0.13,1202.61,0.01,-0.01,0.00,-0.00,-0.0,0.00
4,-0.03,0.01,481.75,-0.00,0.01,0.00,-0.0,-0.00
5,-0.02,-0.01,-0.00,131.09,-0.00,-0.00,0.0,0.00
6,0.00,0.00,0.01,-0.00,34.24,0.00,0.0,0.00
7,-0.00,-0.00,0.00,-0.00,0.00,8.27,-0.0,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,1.8,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,0.32


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00002,0.00000,-0.00001,0.00001,-0.00003
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00005,0.00005,-0.00001,-0.00005
5,-0.00002,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
6,0.00000,0.00001,0.00005,-0.00001,1.00000,0.00002,0.00000,0.00006
7,-0.00001,-0.00001,0.00005,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00001,-0.00001,-0.00001,0.00002,0.00000,-0.00001,1.00000,0.00003
9,-0.00003,0.00000,-0.00005,0.00000,0.00006,-0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18842081857494825

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.67297943e+06 1.78015197e+05 2.75795000e+04 1.96651968e+03
 1.25924140e+02 6.55373640e+00 2.35906566e-01 3.83820618e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999741,-0.021061,-0.008264,-0.002242,-0.000585,-0.000142,-0.000031,-0.000006
1,0.020299,0.996517,-0.078784,-0.017700,-0.004561,-0.001102,-0.000240,-0.000043
2,0.009710,0.077332,0.995036,-0.060085,-0.014369,-0.003448,-0.000752,-0.000135
3,0.003122,0.021807,0.057310,0.995483,-0.070614,-0.015684,-0.003393,-0.000607
4,0.001014,0.007024,0.017582,0.067901,0.993652,-0.085892,-0.017302,-0.003072
5,0.000324,0.002244,0.005593,0.020637,0.081983,0.989693,-0.113870,-0.018941
6,0.000103,0.000718,0.001789,0.006576,0.025219,0.106826,0.977692,-0.178953
7,0.000038,0.000263,0.000657,0.002411,0.009224,0.038213,0.175616,0.983670


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00025864515127971277,
 0.003483208445864916,
 0.004964185228582885,
 0.004517211743546579,
 0.006348295363581058,
 0.010307454692313756,
 0.022308330258007936,
 0.016329689541680126]